# An ECB/CBC detection oracle
Now that you have ECB and CBC working:

Write a function to generate a random AES key; that's just 16 random bytes.

Write a function that encrypts data under an unknown key --- that is, a function that generates a random key and encrypts under it.

The function should look like:
```
encryption_oracle(your-input)
=> [MEANINGLESS JIBBER JABBER]
```
Under the hood, have the function append 5-10 bytes (count chosen randomly) before the plaintext and 5-10 bytes after the plaintext.

Now, have the function choose to encrypt under ECB 1/2 the time, and under CBC the other half (just use random IVs each time for CBC). Use rand(2) to decide which to use.

Detect the block cipher mode the function is using each time. You should end up with a piece of code that, pointed at a block box that might be encrypting ECB or CBC, tells you which one is happening.

In [1]:
from cryptopals import *

In [2]:
def generate_random_bytes(length=16):
    '''Generates a random 16-byte AES key'''
    return bytes(list(np.random.randint(256, size=length)))
#     return ''.join(map(chr, np.random.randint(256, size=length)))

def ECB_CBC_encryption_oracle(plaintext, print_cipher_mode=False):
    '''Oracle that randomly chooses ECB or CBC mode and encrypts the plaintext string with a random key and IV.'''
    prepended_bytes = generate_random_bytes(np.random.randint(5,11))
    appended_bytes = generate_random_bytes(np.random.randint(5,11))
    plaintext = pad_pkcs7(prepended_bytes + to_bytes(plaintext) + appended_bytes)
    
    key, IV = generate_random_bytes(), generate_random_bytes()
    cipher = AES.new(key, AES.MODE_ECB) if np.random.randint(2) == 0 else AES.new(key, AES.MODE_CBC, IV=IV)
    if print_cipher_mode:
        print('MODE_ECB' if cipher.mode == 1 else 'MODE_CBC' if cipher.mode == 2 else 'Other')
    return cipher.encrypt(plaintext)

from collections import Counter
def detect_ECB_CBC_oracle(black_box, BLOCK_SIZE=16):
    '''Detector that determines if a black box is encrypting using ECB or CBC.'''
    long_repeating_string = hex_to_bytes('0'*128)
    
    ciphertext = black_box(long_repeating_string)
    ciphertext_blocks = Counter([ciphertext[i:i+BLOCK_SIZE] for i in range(0,len(ciphertext),BLOCK_SIZE)])
    ct_blocks_are_unique = max(ciphertext_blocks.values()) == 1
    return 'CBC' if ct_blocks_are_unique else 'ECB'

In [3]:
ECB_CBC_encryption_oracle('blah')

b'\x84\x89\x97p]i@"\xd4km\xbfz>|\xe3\xab\xd5)\x0e\xa93\xdf\xf9\xe2Q\x13\x99\x1f\xddk\xc4'

In [4]:
for _ in range(100):
    print(detect_ECB_CBC_oracle(lambda x: ECB_CBC_encryption_oracle(x, print_cipher_mode=True)))
    print()

MODE_CBC
CBC

MODE_CBC
CBC

MODE_CBC
CBC

MODE_CBC
CBC

MODE_ECB
ECB

MODE_CBC
CBC

MODE_ECB
ECB

MODE_ECB
ECB

MODE_ECB
ECB

MODE_CBC
CBC

MODE_CBC
CBC

MODE_CBC
CBC

MODE_ECB
ECB

MODE_ECB
ECB

MODE_ECB
ECB

MODE_ECB
ECB

MODE_CBC
CBC

MODE_CBC
CBC

MODE_CBC
CBC

MODE_ECB
ECB

MODE_ECB
ECB

MODE_CBC
CBC

MODE_CBC
CBC

MODE_ECB
ECB

MODE_ECB
ECB

MODE_ECB
ECB

MODE_CBC
CBC

MODE_CBC
CBC

MODE_ECB
ECB

MODE_CBC
CBC

MODE_CBC
CBC

MODE_CBC
CBC

MODE_CBC
CBC

MODE_ECB
ECB

MODE_ECB
ECB

MODE_ECB
ECB

MODE_CBC
CBC

MODE_CBC
CBC

MODE_ECB
ECB

MODE_ECB
ECB

MODE_ECB
ECB

MODE_CBC
CBC

MODE_ECB
ECB

MODE_ECB
ECB

MODE_CBC
CBC

MODE_CBC
CBC

MODE_CBC
CBC

MODE_CBC
CBC

MODE_ECB
ECB

MODE_ECB
ECB

MODE_ECB
ECB

MODE_CBC
CBC

MODE_CBC
CBC

MODE_CBC
CBC

MODE_ECB
ECB

MODE_ECB
ECB

MODE_CBC
CBC

MODE_CBC
CBC

MODE_ECB
ECB

MODE_ECB
ECB

MODE_ECB
ECB

MODE_ECB
ECB

MODE_ECB
ECB

MODE_CBC
CBC

MODE_CBC
CBC

MODE_CBC
CBC

MODE_ECB
ECB

MODE_CBC
CBC

MODE_CBC
CBC

MODE_ECB
ECB

MODE_CBC
CBC

MODE_C